In [5]:
import pandas as pd
import re
import ast
import json
import redditcleaner

## Import the Reddit posts and comments

In [3]:
df = pd.read_parquet('../chronicPain2020-21.parquet')
df.shape

(354294, 15)

In [4]:
print(len(df[df['body'].isna()]))
df.loc[df['body'].isna(),'body'] = df[df['body'].isna()]['selftext']
print(len(df[df['body'].isna()]))

92287
4835


## Preprocess the text

In [6]:
df = df[df['body'].notna()]
# Clean the markdowns from the text
df['cleaned_body'] = df['body'].map(redditcleaner.clean)

In [92]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import validators

stop_words = list(set(stopwords.words('english')))
stop_words += ["'s", "'t", 'r', "n\'t"]

def preProcess(sentence):
    words = str(sentence).lower()
    words = words.replace('(','').replace(')','')
    words = words.split()
    not_url = [word for word in words if not validators.url(word)]
    without_stop_words = [word for word in not_url if not word in stop_words]
    no_punc = [word for word in without_stop_words if not word in string.punctuation]
    not_one = [word for word in no_punc if not len(word)== 1]
    final =  ' '.join(not_one)
    for punc in string.punctuation:
        final = final.replace(punc,'')
    return final

In [94]:
df['processed'] = df['cleaned_body'].apply(lambda x: preProcess(x))

In [96]:
df = df.drop_duplicates('processed')

279682
279682


## Save the processed text

In [99]:
with open('./pain_nostop.txt', "w") as my_output_file:
    for line in df['processed'].values: 
        my_output_file.write(line+'\n')
    my_output_file.close()

In [100]:
df.to_parquet('./pain_reddit_processed.parquet',engine='fastparquet')